The objective of this notebook is to extract the word cloud of hasthags for each of the events and display them with a given mask and color scheme. The same is also done for the absurd locations. 

In [12]:
import os
import pickle
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import numpy as np
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


In [89]:
# Funciton used to generate and store the wordcloud
def get_prevalent_words(text, event, mask, color = True):

    # read the mask / color image taken from
    # http://jirkavinse.deviantart.com/art/quot-Real-Life-quot-Alice-282261010
    alice_coloring = np.array(Image.open(os.path.join(os.getcwd(),mask)))

    wc = WordCloud(background_color="white", mask=alice_coloring, random_state=42)
    # generate word cloud
    wc.generate(text)

    if color:
        # create coloring from image
        image_colors = ImageColorGenerator(alice_coloring)

    # show
    wc.to_file(os.path.join("PrevalentWords","Hashtags_{}.png".format(event)))
    if color:
        wc.recolor(color_func=image_colors)
        wc.to_file(os.path.join("PrevalentWords","Hashtags_recoloured_{}.png".format(event)))  
      

In [ ]:
# Getting the events folders
events_path = "../../../Project Data/Tweets"
events_folders = os.listdir(events_path)
events_folders =  [x for x in events_folders if "DS_Store" not in x]

# 1. Prevalent Hashtags

In [6]:
event_hashtags = dict()

# For all the events 
for event in events_folders:
    
    # Load the geolocated tweets
    geo_tweets_path = os.path.join(events_path, event, "Geocoded")
    geo_tweets_pkl = os.listdir(geo_tweets_path)
    geo_tweets_pkl = [x for x in geo_tweets_pkl if "Located" in x]
    
    
    event_hasht = list()
    
    print(event)
    for tweet_pkl in tqdm(geo_tweets_pkl):
        tweet_df = pickle.load(open(os.path.join(geo_tweets_path, tweet_pkl), "rb"))

        hashtags = sum(tweet_df["hashtags"].tolist(), [])
        #print(hashtags)
        hashtags = [x.replace("#", "")for x in hashtags if "#" in x[:3]]
        event_hasht += hashtags
        #print(len(event_hasht))
        
    event_hashtags[event] = event_hasht
    
    hashtags=' '.join(event_hasht)
    
    get_prevalent_words(hashtags, event, os.path.join("Masks","tweet_color_white.png"))

pkl = open('event_hashtags.pkl', 'wb')
# Pickle dictionary using protocol 0.
pickle.dump(event_hashtags, pkl)
pkl.close()   

  0%|          | 0/47 [00:00<?, ?it/s]

Bruxelles


  0%|          | 0/333 [00:00<?, ?it/s]

Charlie-Hebdo


  0%|          | 0/28 [00:00<?, ?it/s]

...

  0%|          | 0/221 [00:00<?, ?it/s]

Orlando


  0%|          | 0/18 [00:00<?, ?it/s]

Pakistan


100%|██████████| 18/18 [00:04<00:00,  3.72it/s]


# 2. Observing Abherrant Locations

In [62]:
event_locations = dict()
geolocation_proportions = dict()
location_failures = list()
# For all the events 
for event in events_folders:
    
    # Load the geolocated tweets
    geo_tweets_path = os.path.join(events_path, event, "Geocoded")
    geo_tweets_pkl = os.listdir(geo_tweets_path)
    geo_tweets_pkl = [x for x in geo_tweets_pkl if "Located" in x]
      
    summary = pd.Series([0,0,0,0], index = ["Country/Capital", "City", "Full City Mapping", "Failure"])
    event_locs = list()
    for tweet_pkl in tqdm(geo_tweets_pkl):
        tweet_df = pickle.load(open(os.path.join(geo_tweets_path, tweet_pkl), "rb"))
        event_locs += tweet_df["location"].tolist()
        try:
            summary["Country/Capital"] += (tweet_df["source"].value_counts()["Country/Capital"])
        except:
            print(event, "None in Coutry/Capital")
        try:
            summary["City"] += (tweet_df["source"].value_counts()["City"])
        except:
            print(event, "None in City")
        try:
            summary["Full City Mapping"] += (tweet_df["source"].value_counts()["Full City Mapping"])
        except:
            print(event, "None in Full City Mapping")
        try:
            location_failures += tweet_df.loc[tweet_df['source'].isnull(),'location'].tolist()
            summary["Failure"] += len(tweet_df.loc[tweet_df['source'].isnull(),'location'])
        except :
            print(event, "None in Failure")
    event_locations[event] = event_locs
    
    geolocation_proportions[event] = summary
    hashtags=' '.join(event_locs)

pkl = open('event_locations.pkl', 'wb')
# Pickle dictionary using protocol 0.
pickle.dump(event_locations, pkl)
pkl.close()

pkl = open('location_failures.pkl', 'wb')
# Pickle dictionary using protocol 0.
pickle.dump(location_failures, pkl)
pkl.close()

pkl = open('geolocation_proportions.pkl', 'wb')
# Pickle dictionary using protocol 0.
pickle.dump(geolocation_proportions, pkl)
pkl.close()

 39%|███▉      | 11/28 [00:00<00:00, 106.24it/s]

Istanbul None in City
Istanbul None in Full City Mapping
Istanbul None in City
Istanbul None in Full City Mapping
Istanbul None in City
Istanbul None in Full City Mapping
Istanbul None in Full City Mapping
Istanbul None in City
Istanbul None in Full City Mapping
Istanbul None in City
Istanbul None in Full City Mapping
Istanbul None in Full City Mapping


  0%|          | 0/10 [00:00<?, ?it/s]

Istanbul None in Full City Mapping
Istanbul None in City
Istanbul None in Full City Mapping
Istanbul None in City
Istanbul None in Full City Mapping
Istanbul None in City
Istanbul None in Full City Mapping
Istanbul None in City
Istanbul None in Full City Mapping
Lebanon None in Coutry/Capital
Lebanon None in City
Lebanon None in Full City Mapping


 76%|███████▌  | 168/221 [00:09<00:02, 18.04it/s]

Orlando None in City
Orlando None in Full City Mapping


 84%|████████▎ | 185/221 [00:10<00:01, 18.19it/s]

Orlando None in City
Orlando None in Full City Mapping


100%|██████████| 18/18 [00:00<00:00, 22.67it/s]


In [116]:
for event in geolocation_proportions.keys():
    print(event, sum(geolocation_proportions[event]))
    print(geolocation_proportions[event]/sum(geolocation_proportions[event]))
    print()

Bruxelles 281861
Country/Capital      0.486364
City                 0.314020
Full City Mapping    0.122589
Failure              0.077027
dtype: float64

Charlie-Hebdo 1743964
Country/Capital      0.504950
City                 0.312778
Full City Mapping    0.117664
Failure              0.064609
dtype: float64

Istanbul 64208
Country/Capital      0.467013
City                 0.288624
Full City Mapping    0.135668
Failure              0.108694
dtype: float64

Lebanon 17062
Country/Capital      0.753311
City                 0.078185
Full City Mapping    0.109835
Failure              0.058668
dtype: float64

Nigeria_2015 63384
Country/Capital      0.552852
City                 0.253771
Full City Mapping    0.104916
Failure              0.088461
dtype: float64

Nigeria_2016 8156
Country/Capital      0.540584
City                 0.256498
Full City Mapping    0.104095
Failure              0.098823
dtype: float64

Orlando 1277929
Country/Capital      0.337963
City                 0.435751
Ful

In [ ]:
get_prevalent_words(hashtags, event, os.path.join("Masks","tweet_color_white.png"))

Manually selecting a certain number of tweets which were not geolocalised for visualization. 

In [103]:
funny_locations = ['she/her','Underground','Londonnnn','my imagination',
                   'Étudiant en Infographie','scorpio//slytherin',
                   "Who's asking",'Worldwide','#JusticeMatters',
 'in my ladymountain™ gc','Global','Global Cityzen',
 'Uk','Heavenly Chances','woodworld','irgendwo im nirgendwo',
 'Worlwide','Denial','indoors','Somewere','F.W.I',
 'borderline bipolar','Wherever Rihanna Is','In your Nightmares',
 'Finding My Purpose',
 'Redacted, Omited','earthlings','Au calme','::loading::',
 'Multiverse.',"In God's hands","My mind's alternate dimensions",
 'Global','Global','Global','My World','Worldwide Promotion',
 'Winterfell','inactive','hogwarts','Neverland :)','he followed me',
 'inactive','Global','Partout','In Wonderlaaaaaaaaaand',
 '#TheHurtingTheHealingTheLoving','Neverland',
 'inactive',
 'partout','partout',"Knocking on heaven's door",'In Flight',
 'Worldwide','an airplane','localhost','Depends','En la superficie',
 'In my world','Online','hogwarts','Shondaland','World','militante',
 'Omniscient','429 km','Poudlard','hogwarts','World','Undefined',
 '#LOLOTOR','WORLDWIDE','hogwarts','blinks','Hitchhiking',
 'Your nightmare','Hogwarts',"N 47°27' 0'' / W 122°16' 0''",'hogwarts',
 'On my journey.',"A l'ouest.",'\xa0👻 : littlemarinee',
 'À droite','I tweet therefore I am',"tu m'suis",
 '#BeeChallenge','Always \xa0🐍','Worldwide','Worldwide',
 'Worldwide','Worldwide','Underground','▬ Anywhere ▬','Hogwarts',
 '\xa0🌍','Dancing with Delilah','#Global','Hogwarts',
 'Hogwarts','Hogwarts','Global','she/her','Inbetween',
 'Worldwide','#SouthOfFrance','World','WORLD','#Astronaut ∞',
 '#LulzLand','Omnipresent','On my journey.','No where',
 'On my journey.','worldwide','#WOMENAREUROPE','Upside Down',
 'Neverland','poudlard | 43','Hogwarts','World',
 'UK','On my journey.','World',
 'Anywhere','Global','@WhatAboutWolves',
 '#Global','I tweet therefore I am','@Internet',
 'in my world','Global','Quatrième neurone à gauche','Pays du surréalisme \xa0🇧🇪',
 'he/him','On my journey.','Hogwarts','On my journey.',
 'hogwarts','Makeourplanetgreatagain','Worldwide','Hogwart',
 'HURTS','Underground','On Facebook at','CityofTheAngels',
 'Probably traveling','On Facebook at','Anywhere','#Citizenoftheworld',
 'On my journey.','Neverland \xa0♥','Hogwarts','Anywhere',
 'Hogwarts','Worldwide','Winterfell ☾ | Hogwarts',
 '29','On my journey.',"Britney's world",'World','Principle Expertise',
 'ЯESISTANT','Cyberspace','CyberSpace','Rosewood/Ravenswood',
 '...','CheetahCave','Omnipresent','Truth or Consequences, NM',
 'Online','Follow me on Instagram','Hogwarts','Twittsphere',
  'Neverland', 'Hogwarts','#ThankYouIOI','he/him',
 '420', 'Worldwide','Pays du surréalisme \xa0🇧🇪',
 '• lt Is What It Is •','goldenworld',
'Worldwide','Hogwarts','Online','Au Nooord','Underground',
 'World','Global','Global','Ravenclaw | Storybrooke','GAGALAND',
 'World','Poudlard','Exceptionalistan','neverland',
 'Worldwide','Worldwide','Heartland','|Hogwarts|','Humanité',
 'Worldwide','Winterfell.','Worldwide','Derrière un écran',
 'World','256 area_code','Neverland','Worldwide','Tatooine',
 'Hogwarts','Hogwarts','Worldwide','Hellfire','Poudlard',
 'World','Global','Moonland','Internet','Internet',
 'Worldwide','Traveling','Multiverse','Worldwide',
 'Global','Worldwide','Eurozone','worldwide',
 '☼·• ↦◦↤ ∘ ◯ ∅ ●●',
 'Worldwide','Worldwide',
 '#Anonymous','global',"Migrante",'neverland','twitter curator','Worldwide',
 'Migrante','worldwide','Poudlard','Là où il me plaît !','Reality/Fantasy',
 'lalaland','underground','worldwide','worldwide','Nanananaaa','Floriduh'
 'UK','World','Could be anywhere','#verNederland',"God's Hands",
 'worldwide','Global','Worldwide','Global','RESiSTANBUL','In a crazy world.',
 'worldwide','Global','worldwide','Global','Worldwide','Winterfell\xa0✨',
 '127.0.0.1','51.3 NB en 5.5 OL','\xa0☣ Grown Up Punk \xa0☣',
 'world','world','myworld\xa0♥','In a crazy world.','World',
 'Worldwide','Hogwarts','HoneymoonTour','Worldwide',
 'WORLDWIDE','Hogwarts','Global','In your thought',
 'Worldwide','dreamville','global','#LibertéJeChérisTonNom','Online', 'World', "global",'CyberSpace', 'Hogwarts',
                  'WORLDWIDE.','Global','WORLDWIDE.',"ailleurs… \xa0✨",
                  'Offices worldwide','facebook subyroy','facebook subyroy',
                 'Global', 'warning!somegraphicphotosnews',
 'World',  'Là où il me plaît !', "Worldwide",
 'RevolutionFrontLines',
 'Worldwide',
 'Tweeting in 5 languages', 'Abaddon','Beside Myself...',
 'Worldwide','Hellgium',
 'World',
 'Hellgium',
 'Weimerica',
 'Worldwide','LilyAmis.blogspot.com',
 'Worldwide',
 'WorldWide !', 'Where I am needed!', 'Worldwide',
 'Worldwide',
 '( ͡° ͜ʖ ͡°)','#FollowTheWhiteRabbit','Multiverse',
 '50.430482,4.500769',
 'Erdenbewohner',
 '( ͡° ͜ʖ ͡°)',
 'Worldwide',
 '( ͡° ͜ʖ ͡°)','( ͡° ͜ʖ ͡°)',
 'World news أخبار العالم',
 'World', "Women's rights",
                  'Winterfell',"In Your Atmosphere",
 "It's classified",
 'Worldwide',"LarryLand™",
 '10/11/13',
 '10/11/13',
 'WorldWide !',"WorldWide !",
 'worldwide (ﾉ◕ヮ◕)ﾉ*:･ﾟ✧', 'Worldwide','I am ubiquitous!',
 'Worldwide',
 'Global','Là où il me plaît !',
 'Là où il me plaît !',
 'Là où il me plaît !','#9ll=#NWO #Nuclear #FalseFlag!',
 '#9ll=#NWO #Nuclear #FalseFlag!',
 '#9ll=#NWO #Nuclear #FalseFlag!',
 'Là où il me plaît !',
 'Là où il me plaît !',
 'Global',
 'Là où il me plaît !','Until Whenever With BIGBANG','Worldwide','Undergroud',
 'Neverland',
 'Worldwide']

funny_locations = [x.replace(" ", "").replace(",","") for x in funny_locations]

print(len(set(funny_locations)))
get_prevalent_words(" ".join(funny_locations), "_locations_", os.path.join("Masks","facepalm_mask.png"), color = False)   


359
195


In [114]:

location_failures

['World news أخبار العالم',
 'Barihoel',
 'Online',
 '38.257025, 140.878905',
 'World',
 'FL NC GA',
 'fdvlh',
 'surrounded by dumbasses',
 'surrounded by dumbasses',
 'Classified',
 'Twitter, FB, Insta...and DC',
 '20.606953,-87.091285',
 'I am Omnipresent.',
 'Global',
 'Skype+Email dj2015@outlook.com',
 'bretigny(Essonne)',
 'TN',
 'j.w.d.',
 'Global',
 'Twitter, FB, Insta...and DC',
 'DC-SATX',
 'fb.com/demokratischeseuropa',
 'deutscher Sprachraum',
 'Publizist & Blogger',
 'Publizist & Blogger',
 'Là où il me plaît !',
 'Publizist & Blogger',
 'Oslo/Nesodden',
 'Hamburg/Brussels',
 'TN',
 'zandbak geboortedatum +/- 570',
 'CDMX',
 'US',
 'Loemek',
 'global',
 'Boston/Mattapan, MA',
 'with my fiancé',
 'CyberSpace',
 'EU',
 'CyberSpace',
 'Leeds/UK',
 'Hogwarts',
 '\xa0✉ margot@aheartyhome.com',
 'US',
 'Reality',
 'WBUR, stations nationwide',
 "'Nooga\xa0✈️Mannheim\xa0✈️K'ville",
 'www.facebook.com/bowenswharf',
 'Davis.Oakland.LA.NYC.',
 'WORLDWIDE.',
 'Global',
 'WORLDWIDE.',
 